<div id="TOP" />
<a href="#END" > END of Notebook (filter) </a>

# TODO

map_containers = {
   "c1": {
       "image": "mjbright/k8s-demo:1", 
        "extport": 8200,
         ...] ,
   "c2": ...

}

In [1]:
# -- CONFIG ------------------------------------
LAB_TAG=${PWD##*/} 
LAB_NAME="TFI_Azure_Lab4c.Docker"
LAB_WEIGHT=44
LAB_MODE="Terraform"
export MAX_LINE_LEN=80
# ----------------------------------------------

DIE() { echo "[$PWD] Error: $*"; banner "ERROR"; exit 1; }
NBTOOL_RC=~/scripts/nbtool.rc; [ ! -f $NBTOOL_RC ] && DIE "No such script as $NBTOOL_RC"
echo "==== Reading nbtool functions from '$NBTOOL_RC' ===="; . $NBTOOL_RC
EXCL_FN_INIT_NOTEBOOK
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

==== Reading nbtool functions from '/home/student/scripts/nbtool.rc' ====
TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.

******** Mon 09 Jan 2023 05:32:11 PM UTC: Notebook initialized on tf TF-1.3.7

NB_DIR=/home/student/src/mjbright.tf-scenarios-private/ServeU

#EXCLUDE
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

NB_DIR=$PWD
rm -rf ~/tmp/terraform*
#rm -rf ~/labs

FRONTMATTER=$(cat <<END_HEREDOC
---
title: 'Lab 4c.Docker'
date: 2019-02-11T19:27:37+10:00
weight: 43
---

END_HEREDOC
)

echo; echo "==== Reading nbtool functions ===="
. ~/bin/nbtool.rc

echo; echo "==== Setting Exercise tag ===="
LABTAG=${PWD##*/}  # LABTAG="4c.Docker"
echo "LABTAG=$LAB_TAG" | EXCL_FN_HIGHLIGHT_EOL =

echo; echo "==== Setting TF_DATA_DIR ===="
export TF_DATA_DIR=~/dot.terraform; echo "TF_DATA_DIR=$TF_DATA_DIR" | EXCL_FN_HIGHLIGHT_EOL =

NO_EXEC echo SHOULD NOT BE SEEN | EXCL_FN_HIGHLIGHT_SOL SHOULD
EXEC    echo SHOULD BE SEEN     | EXCL_FN_HIGHLIGHT_SOL SHOULD

# echo; # echo "==== Filtering current notebook ===="; EXCL_FN_FILTER_NOTEBOOK index.ipynb

echo; echo "==== Setting AWS vars for student20 account ===="
. ~/.aws/linked/student20.rc; env | grep MYAWS_ACCOUNT_NAME= | EXCL_FN_HIGHLIGHT_EOL =

#  --INCLUDE--SECTION--

In [25]:
EXCL_FN_LAB_ENV 0

2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

## Background:

Here, we will investigate the use of another Provider - the Docker Provider - which will allow us to quickly deploy configurations

You can find information about the Provider here: https://registry.terraform.io/providers/kreuzwerker/docker/latest/docs

## Tasks:
### 1. Make a directory called ‘lab4c’ underneath the labs directory.
### 2. Change into the directory.
### 3. Create the following files: main.tf, versions.tf

# 4c.1 Create a "docker_container" resource

In [17]:
#EXCLUDE:

mkdir -p ~/labs/lab4c
cd ~/labs/lab4c

export TF_VAR_user=student20
export TF_VAR_port0=8200

cat > versions.tf <<EOF

terraform {
  required_providers {
    docker = {
      #source = "terraform-providers/docker"
      source = "kreuzwerker/docker"
    }
  }
  required_version = ">= 1.0"
}  

provider "docker" {
  host = "unix:///var/run/docker.sock"
}

EOF

cat > main.tf <<EOF
variable "user" {
    description = "A unique user-name to be used as part of container names"
}

variable "port0" {
    description = "A unique starting port to be used to expose our containers"
}

resource "docker_container" "docker_app" {
  image = "mjbright/docker-demo:1"

  name  = "\${var.user}_docker_app"
  restart = "always"

  volumes {
    container_path  = "/tmp"
    # replace the host_path with full path for your project directory starting from root directory /
    host_path = "/tmp/demo"
    read_only = false
  }

  ports {
    internal = 80
    external = var.port0
  }
  
  # not working? tags = { LabName = "$LAB_TAG" }
}    
EOF

## 4c.1.1. Configure Terraform and Docker Providers

We will first define the version of Terraform and the Docker Provider

The versions.tf file should contain:

In [18]:
EXEC cat versions.tf


terraform {
  required_providers {
    docker = {
      #source = "terraform-providers/docker"
      source = "kreuzwerker/docker"
    }
  }
  required_version = ">= 1.0"
}  

provider "docker" {
  host = "unix:///var/run/docker.sock"
}



## 4c.1.2. Configure the Docker container resource

We will create a single running Docker container using a test Docker image *mjbright/docker-demo:1*

The Docker container will listen on Port 80 (internally), we will expose this at Port 8080

Note: We define a Volume, but we will not actually use this in the application

The main.tf file should contain:


In [4]:
EXEC cat main.tf

variable "user" {
    description = "A unique user-name to be used as part of container names"
}

variable "port0" {
    description = "A unique starting port to be used to expose our containers"
}

resource "docker_container" "docker_app" {
  image = "mjbright/docker-demo:1"

  name  = "${var.user}_docker_app"
  restart = "always"

  volumes {
    container_path  = "/tmp"
    # replace the host_path with full path for your project directory starting from root directory /
    host_path = "/tmp/demo"
    read_only = false
  }

  ports {
    internal = 80
    external = var.port0
  }
  
  # not working? tags = { LabName = "4c.Docker" }
}    


# 4c.2 Apply the config

## 4c.2.1 Check for any running containers

First check for any running containers using ```docker ps```

There may be some, hopefully they won't be listening on the Port you intend to use (TF_VAR_port0)

## 4c.2.2 Apply your config

Apply the config and you should notice that the Docker container is created very quickly

In [20]:
#EXCLUDE:
TF_DESTROY
TF_INIT
TF_APPLY

docker_container.docker_app[4]: Refreshing state... [id=3c810773ebef372e73917a05005d74b02fe3a1e885db68c88adae610bc1e1dc9]
docker_container.docker_app[1]: Refreshing state... [id=5ddecdeda58b866515dedbd815992de3aa4d46cda3662e7303db7062cef28578]
docker_container.docker_app[2]: Refreshing state... [id=3d5531371df48540ad616316bd2169cc801aefd06d6a9ac103605d57f983b981]
docker_container.docker_app[5]: Refreshing state... [id=44ee65003f0f57b007c7b07d26ef59cf341dfc6abc3bf925ccfd1f6a939152e1]
docker_container.docker_app: Refreshing state... [id=6d0f222d8acb0ed16b097be3229f1f85651f02ad19ca834fd034accfe21dbcca]
docker_container.docker_app[3]: Refreshing state... [id=2e10bd6dcf0317464d2a9643a92ec2a3e2f99adcf0f17421f7d4d41ef08c83aa]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # docker_container.docker_app will be destroyed
  - resource "docker_

          - protocol = "tcp" -> null
        }

      - volumes {
          - container_path = "/tmp" -> null
          - host_path      = "/tmp/demo" -> null
          - read_only      = false -> null
        }
    }

  # docker_container.docker_app[1] will be destroyed
  - resource "docker_container" "docker_app" {
      - attach                                      = false -> null
      - command                                     = [
          - "/app/demo-binary",
          - "-l",
          - "80",
          - "-L",
          - "0",
          - "-R",
          - "0",
        ] -> null
      - container_read_refresh_timeout_milliseconds = 15000 -> null
      - cpu_shares                                  = 0 -> null
      - dns                                         = [] -> null
      - dns_opts                                    = [] -> null
      - dns_search                                  = [] -> null
      - entrypoint                                  = [] -> null
      - e

      - attach                                      = false -> null
      - command                                     = [
          - "/app/demo-binary",
          - "-l",
          - "80",
          - "-L",
          - "0",
          - "-R",
          - "0",
        ] -> null
      - container_read_refresh_timeout_milliseconds = 15000 -> null
      - cpu_shares                                  = 0 -> null
      - dns                                         = [] -> null
      - dns_opts                                    = [] -> null
      - dns_search                                  = [] -> null
      - entrypoint                                  = [] -> null
      - env                                         = [] -> null
      - gateway                                     = "172.17.0.1" -> null
      - group_add                                   = [] -> null
      - hostname                                    = "3d5531371df4" -> null
      - id                                    

      - cpu_shares                                  = 0 -> null
      - dns                                         = [] -> null
      - dns_opts                                    = [] -> null
      - dns_search                                  = [] -> null
      - entrypoint                                  = [] -> null
      - env                                         = [] -> null
      - gateway                                     = "172.17.0.1" -> null
      - group_add                                   = [] -> null
      - hostname                                    = "2e10bd6dcf03" -> null
      - id                                          = "2e10bd6dcf0317464d2a9643a92ec2a3e2f99adcf0f17421f7d4d41ef08c83aa" -> null
      - image                                       = "sha256:3374313a743006d37984b6818d4c18df15ab4e6432ea1ae189791d2c36fb3dd8" -> null
      - init                                        = false -> null
      - ip_address                                  = "172.17

      - env                                         = [] -> null
      - gateway                                     = "172.17.0.1" -> null
      - group_add                                   = [] -> null
      - hostname                                    = "3c810773ebef" -> null
      - id                                          = "3c810773ebef372e73917a05005d74b02fe3a1e885db68c88adae610bc1e1dc9" -> null
      - image                                       = "sha256:3374313a743006d37984b6818d4c18df15ab4e6432ea1ae189791d2c36fb3dd8" -> null
      - init                                        = false -> null
      - ip_address                                  = "172.17.0.7" -> null
      - ip_prefix_length                            = 16 -> null
      - ipc_mode                                    = "private" -> null
      - links                                       = [] -> null
      - log_driver                                  = "json-file" -> null
      - log_opts                  

      - image                                       = "sha256:3374313a743006d37984b6818d4c18df15ab4e6432ea1ae189791d2c36fb3dd8" -> null
      - init                                        = false -> null
      - ip_address                                  = "172.17.0.5" -> null
      - ip_prefix_length                            = 16 -> null
      - ipc_mode                                    = "private" -> null
      - links                                       = [] -> null
      - log_driver                                  = "json-file" -> null
      - log_opts                                    = {} -> null
      - logs                                        = false -> null
      - max_retry_count                             = 0 -> null
      - memory                                      = 0 -> null
      - memory_swap                                 = 0 -> null
      - must_run                                    = true -> null
      - name                                        =

      + command                                     = (known after apply)
      + container_logs                              = (known after apply)
      + container_read_refresh_timeout_milliseconds = 15000
      + entrypoint                                  = (known after apply)
      + env                                         = (known after apply)
      + exit_code                                   = (known after apply)
      + gateway                                     = (known after apply)
      + hostname                                    = (known after apply)
      + id                                          = (known after apply)
      + image                                       = "mjbright/docker-demo:1"
      + init                                        = (known after apply)
      + ip_address                                  = (known after apply)
      + ip_prefix_length                            = (known after apply)
      + ipc_mode                               

## 4c.2.3 Check for the running container

Check for your container using ```docker ps```

If you are student20 for example you should see a container called student20_docker_app exposed on port 8020:

In [21]:
docker ps

CONTAINER ID   IMAGE                    COMMAND                  CREATED          STATUS          PORTS                  NAMES
1cfb2ba43d2a   mjbright/docker-demo:1   "/app/demo-binary -l…"   11 seconds ago   Up 10 seconds   0.0.0.0:8200->80/tcp   student20_docker_app


## 4c.2.4 Curl the container

In [22]:
curl 127.0.0.1:8200


                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
                             .:::///:::///+++///:::///:::.                    `.ossss:.             
                             .:///////:/+++oo++/:///////:.                    .-ssssss:.            
                    .-:::::::--///////--:::::::--///////--:::::::-.           `-sssssso..

### 4c.3 Create 6 containers

First destroy the previous container using Terraform

Then modify your main.tf to now create 6 containers running image

- mjbright/docker-demo:1
- mjbright/docker-demo:2
- etc ...

**Note**: They will each need to have a unique name and to expose on a unique external Port

**Note**: If you remove the ```external``` line in the ```port``` block, Docker will automatically assign a non-conflicting port to each container.

### 4c.4 Cleanup

In [23]:
NO_EXEC terraform destroy
EXEC terraform destroy --auto-approve

docker_container.docker_app: Refreshing state... [id=1cfb2ba43d2a5ca4994608535f9a6a5904befc7ef74fc7be209a3cae04ea9452]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # docker_container.docker_app will be destroyed
  - resource "docker_container" "docker_app" {
      - attach                                      = false -> null
      - command                                     = [
          - "/app/demo-binary",
          - "-l",
          - "80",
          - "-L",
          - "0",
          - "-R",
          - "0",
        ] -> null
      - container_read_refresh_timeout_milliseconds = 15000 -> null
      - cpu_shares                                  = 0 -> null
      - dns                                         = [] -> null
      - dns_opts                                    = [] -> null
      - dns_search                        


Destroy complete! Resources: 1 destroyed.



To destroy the containers

<hr/>

# Summary

In this Exercise we looked at the use of another Provider - Docker

We then
1. Created a single Docker container exposing it's web server on port 8080
2. Created multiple Docker containers exposing their web servers on port 8080, 8081, etc ...

**Note**: The Docker Provider is very quick as it acts locally - this can be useful to able to test out Terraform concepts/syntax without the delays and costs of using a distant Cloud Provider.

In [24]:
EXCL_FN_LAB_ENV 0

2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg) -->
<img src="../images/LOGO_v2_CROPPED.jpg" width="200" />

# Pragma --EXCLUDE--SECTION--

## **NOTE:** Save notebook first

<div id="END" /> <a href="#TOP" > TOP OF PAGE </a>  <!-- must appear after EXCLUDE pragma !! -->

In [26]:
set +x
. ~/scripts/nbtool.rc
pwd
#NB_DIR=$PWD
#echo NB=$NB
#echo NB_DIR=$NB_DIR
#set -x
set +x
EXCL_FN_FILTER_NOTEBOOK $NB

TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
/home/student/labs/lab4c

==== Filtering current notebook [/home/student/src/mjbright.tf-scenarios-private/ServeUpLabs/content/tf-labs/4c.Docker/README.ipynb] ====

******** Mon 09 Jan 2023 05:51:08 PM UTC: Notebook

In [2]:
exit 1

exit
Restarting Bash

In [11]:
#EXCLUDE:

mkdir -p ~/labs/lab4c
cd ~/labs/lab4c

export TF_VAR_user=student20
export TF_VAR_port0=8200

cat > versions.tf <<EOF

terraform {
  required_providers {
    docker = {
      #source = "terraform-providers/docker"
      source = "kreuzwerker/docker"
    }
  }
  required_version = ">= 1.0"
}  

provider "docker" {
  host = "unix:///var/run/docker.sock"
}

EOF

cat > main.tf <<EOF
variable "user" {
    description = "A unique user-name to be used as part of container names"
}

variable "port0" {
    description = "A unique starting port to be used to expose our containers"
}


resource "docker_container" "docker_app" {
  count = 6
  image = "mjbright/k8s-demo:${ count.index + 1 }"

  name  = "\${var.user}_k8s_app_idx\${count.index + 1}"
  restart = "always"

  volumes {
    container_path  = "/tmp"
    # replace the host_path with full path for your project directory starting from root directory /
    host_path = "/tmp/demo"
    read_only = false
  }

  ports {
    internal = 80
    #external = var.port0
  }
  
  # not working? tags = { LabName = "$LAB_TAG" }
}    
EOF

In [12]:
TF_APPLY

docker_container.docker_app[1]: Refreshing state... [id=0fe8810a15e6b6bc59f5cb42d5e9d75dce504369c4bb6d1ef80cb7b741ea0329]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create
-/+ destroy and then create replacement

Terraform will perform the following actions:

  # docker_container.docker_app[0] will be created
  + resource "docker_container" "docker_app" {
      + attach                                      = false
      + bridge                                      = (known after apply)
      + command                                     = (known after apply)
      + container_logs                              = (known after apply)
      + container_read_refresh_timeout_milliseconds = 15000
      + entrypoint                                  = (known after apply)
      + env                                         = (known after apply)
      + exit_code                                  

      - log_opts                                    = {} -> null
      - max_retry_count                             = 0 -> null
      - memory                                      = 0 -> null
      - memory_swap                                 = 0 -> null
      ~ name                                        = "student20_k8s_app" -> "student20_k8s_app_idx1" # forces replacement
      ~ network_data                                = [
          - {
              - gateway                   = "172.17.0.1"
              - global_ipv6_address       = ""
              - global_ipv6_prefix_length = 0
              - ip_address                = "172.17.0.2"
              - ip_prefix_length          = 16
              - ipv6_gateway              = ""
              - network_name              = "bridge"
            },
        ] -> (known after apply)
      - network_mode                                = "default" -> null
      - privileged                                  = false -> null
      - 

          + read_only      = false
        }
    }

  # docker_container.docker_app[3] will be created
  + resource "docker_container" "docker_app" {
      + attach                                      = false
      + bridge                                      = (known after apply)
      + command                                     = (known after apply)
      + container_logs                              = (known after apply)
      + container_read_refresh_timeout_milliseconds = 15000
      + entrypoint                                  = (known after apply)
      + env                                         = (known after apply)
      + exit_code                                   = (known after apply)
      + gateway                                     = (known after apply)
      + hostname                                    = (known after apply)
      + id                                          = (known after apply)
      + image                                       = "mjbright/

      + wait_timeout                                = 60

      + healthcheck {
          + interval     = (known after apply)
          + retries      = (known after apply)
          + start_period = (known after apply)
          + test         = (known after apply)
          + timeout      = (known after apply)
        }

      + labels {
          + label = (known after apply)
          + value = (known after apply)
        }

      + ports {
          + external = (known after apply)
          + internal = 80
          + ip       = "0.0.0.0"
          + protocol = "tcp"
        }

      + volumes {
          + container_path = "/tmp"
          + host_path      = "/tmp/demo"
          + read_only      = false
        }
    }

  # docker_container.docker_app[5] will be created
  + resource "docker_container" "docker_app" {
      + attach                                      = false
      + bridge                                      = (known after apply)
      + command              

In [13]:
docker ps

CONTAINER ID   IMAGE                 COMMAND                  CREATED          STATUS          PORTS                   NAMES
be5de0154136   mjbright/k8s-demo:1   "/app/demo-binary -l…"   10 seconds ago   Up 9 seconds    0.0.0.0:32774->80/tcp   student20_k8s_app_idx1
b2f5ac9c28d2   mjbright/k8s-demo:1   "/app/demo-binary -l…"   11 seconds ago   Up 10 seconds   0.0.0.0:32771->80/tcp   student20_k8s_app_idx5
cd306a2094e5   mjbright/k8s-demo:1   "/app/demo-binary -l…"   11 seconds ago   Up 9 seconds    0.0.0.0:32772->80/tcp   student20_k8s_app_idx0
c4c8426fd4f7   mjbright/k8s-demo:1   "/app/demo-binary -l…"   11 seconds ago   Up 9 seconds    0.0.0.0:32773->80/tcp   student20_k8s_app_idx3
faef86207fd1   mjbright/k8s-demo:1   "/app/demo-binary -l…"   11 seconds ago   Up 10 seconds   0.0.0.0:32770->80/tcp   student20_k8s_app_idx2
ba4281fded55   mjbright/k8s-demo:1   "/app/demo-binary -l…"   11 seconds ago   Up 10 seconds   0.0.0.0:32769->80/tcp   student20_k8s_app_idx4


In [22]:
cd $NB_DIR
nbtool.py -f index.ipynb
ls -altr *.ipynb

jupyter-nbconvert --to markdown index.ipynb.filtered.ipynb

echo "$FRONTMATTER"          > index.md
cat index.ipynb.filtered.md >> index.md
mv  index.ipynb.filtered.md index.ipynb.filtered.md.bak
ls -altr *.md

cells to include[#15]=[[2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18]]
cells to include[#15]=[[18, 17, 16, 15, 14, 13, 12, 10, 9, 8, 7, 6, 5, 4, 2]]
-rw-rw-r-- 1 student docker 38557 Oct  8 16:31 index.ipynb
-rw-rw-r-- 1 student docker 16720 Oct  8 16:31 index.ipynb.filtered.ipynb
[NbConvertApp] Converting notebook index.ipynb.filtered.ipynb to markdown
[NbConvertApp] Writing 12891 bytes to index.ipynb.filtered.md
-rw-rw-r-- 1 student docker    75 Feb  2  2021 HEADER.md
-rw-rw-r-- 1 student docker 13129 Oct  8 16:31 index.md
